In [1]:
import docx

In [21]:
doc = docx.Document('../data/123.docx')

In [22]:
for table in doc.tables:
    print (table.rows[0].cells[0].text)

ФИО
Общие анализы крови
Биохимические анализы крови
Общие анализы мочи
Иммунологические исследования
Дополнительные исследования
Микробиологические посевы
Этиотропная терапия
Температурный лист


In [40]:
tables_mapping = {'ФИО': 'general_info', 
                  'Общие анализы крови': 'cbc',
                  'Биохимические анализы крови': 'Biochem',
                  'Общие анализы мочи': 'urine',
                  'Иммунологические исследования': 'immunology',
                  'Дополнительные исследования': 'additional',
                  'Микробиологические посевы': 'microbiology',
                  'Этиотропная терапия': 'antibiotics',
                  'Температурный лист': 'temperature'}

In [43]:
tables = {}
for table in doc.tables:
    tables[tables_mapping[table.rows[0].cells[0].text.strip()]] = table
tables

{'general_info': <docx.table.Table at 0x7f2d4fb8d7b8>,
 'cbc': <docx.table.Table at 0x7f2d4fb8d630>,
 'Biochem': <docx.table.Table at 0x7f2d4fb8dd68>,
 'urine': <docx.table.Table at 0x7f2d4fb8d1d0>,
 'immunology': <docx.table.Table at 0x7f2d4fb8d160>,
 'additional': <docx.table.Table at 0x7f2d4fb8dfd0>,
 'microbiology': <docx.table.Table at 0x7f2d4fb8d5f8>,
 'antibiotics': <docx.table.Table at 0x7f2d4fb8dcf8>,
 'temperature': <docx.table.Table at 0x7f2d4fb8d358>}

In [41]:
#get temperature data
temperature = {}
the_table = tables['temperature']
rows_num = len(the_table.rows)
cells_num = len(the_table.rows[1].cells)
temp_id = 0
for row in range(1, rows_num, 3):
    for cell in range(0, cells_num):
        date = the_table.rows[row].cells[cell].text
        if date != '':
            temperature[temp_id] = {'date':date,
                                    'temp': the_table.rows[row+1].cells[cell].text}
            temp_id+=1
        else: 
            break

In [37]:
#get general info
gen_info_mapping = {'ФИО': 'name',
                    'Дата рождения': 'birthdate',
                    'Адрес домашний': 'address',
                    'Отделение': 'department',
                    'Дата госпитализации': 'admission_date',
                    'Дата выписки': 'discharge_date',
                    'Клинический диагноз': 'diagnosis'}
general_info = {}
the_table = tables['general_info']
for row in the_table.rows:
    general_info[gen_info_mapping[row.cells[0].text.strip()]] = row.cells[1].text
general_info

{'name': 'ЙЦУ',
 'birthdate': '11.11.1988',
 'address': 'ФЫВА.',
 'department': 'РАО -> 2 хирургическое',
 'admission_date': '04.09.2018',
 'discharge_date': '05.12.2018',
 'diagnosis': 'ЖКБ. Хронический калькулезный холецистит. Холедохолитиаз. Механическая желтуха. Острый некротический билиарный панкреатит. Инфицированный панкреонекроз. Гнойный оментобурсит. Парапанкреальная флегмона слева. Некроз брыжейки поперечно-ободочной кишки. Острая язва желудка, осложненная кровотечением от 11.09.2018г. Сепсис. Сепстический шок от 07.09.2018г. Левосторонний реактивный. Полифакторная анемия средней степени. Первичный гипотиреоз, ремиссия.'}

In [70]:
#get CBC info
the_table = tables['cbc']
cbc_units = {}
cbc_data = {}
rows_num = len(the_table.rows)
cells_num = len(the_table.rows[1].cells)
blank_rows_ind = []

for i in range(rows_num):
    row = the_table.rows[i]
    if (row.cells[0].text == '' and row.cells[1].text == ''):
        blank_rows_ind.append(i)
blank_rows_ind.append(len(the_table.rows))
        
# for row in range(1, rows_num):
#     unit_name = the_table.rows[row].cells[0].text
#     if unit_name != '':
#         cbc_units[the_table.rows[row].cells[0].text] = the_table.rows[row].cells[1].text
#     else: 
#         continue

start = 0
for blank in blank_rows_ind:
    for cell in range(2, cells_num):
        date = the_table.rows[start+1].cells[cell].text.strip()
        if date == '':
            continue
        analysis = {}
        for row in range (start+2, blank):
            observation = the_table.rows[row].cells[cell].text.strip()
            if observation != '':
                analysis[the_table.rows[row].cells[0].text.strip()] = observation
            else: 
                continue
        cbc_data[date] = analysis
    start = blank


In [71]:
cbc_data

{'04.09': {'RBC': '3.80',
  'Hgb': '161',
  'Hct': '44.3',
  'Tr': '281',
  'WBC': '22.17',
  'п/яд': '5',
  'с/яд': '81',
  'лимф': '10',
  'мон': '3',
  'эоз': '1',
  'СОЭ': '45'},
 '05.09': {'RBC': '4.10',
  'Hgb': '134',
  'Hct': '36.7',
  'Tr': '286',
  'WBC': '17.02',
  'п/яд': '5',
  'с/яд': '86',
  'лимф': '4',
  'мон': '5',
  'СОЭ': '8'},
 '06.09': {'RBC': '3.92',
  'Hgb': '127',
  'Hct': '35.9',
  'Tr': '271',
  'WBC': '16.18',
  'п/яд': '9',
  'с/яд': '77',
  'лимф': '3',
  'мон': '10',
  'эоз': '1',
  'СОЭ': '10',
  'ТЗН': '++'},
 '07.09': {'RBC': '3.42',
  'Hgb': '111',
  'Hct': '31.8',
  'Tr': '274',
  'WBC': '17.71',
  'п/яд': '6',
  'с/яд': '80',
  'лимф': '7',
  'мон': '6',
  'эоз': '1',
  'СОЭ': '15'},
 '08.09': {'RBC': '2.66',
  'Hgb': '87',
  'Hct': '25.5',
  'Tr': '230',
  'WBC': '42.82',
  'юные': '5',
  'п/яд': '15',
  'с/яд': '76',
  'лимф': '2',
  'мон': '2',
  'СОЭ': '22',
  'ТЗН': '++++'},
 '09.09': {'RBC': '3.17',
  'Hgb': '102',
  'Hct': '30.2',
  'Tr': '11